1) You previously created an inverted index of terms and pages; this time, create a dictionary that contains an inverted index from the values contained in web_graph_small.txt. Assume the number in the first column is linking to the second. 

You can set either the first number as the key and all nodes it links to as a list of values, or the other way around -- just be sure you note explicitly which it is in comments to keep track.  In fact, some people find it easier for calculation to make two inverted indexes (one for nodes:outlinks, one for nodes:inlinks, but you do you).

In [1]:
from nltk import flatten

In [2]:
import sys
from random import randrange
sys.setrecursionlimit(randrange(2500, 3000))

In [3]:
with open ('web_graph_small.txt','r') as file:
    content = [line.strip().split('\t') for line in file.readlines()]

In [4]:
out_degree = {}
for (key, value) in content:
    out_degree.setdefault(key, []).append(value)
out_degree

{'0': ['3', '2', '4', '5'],
 '3': ['0',
  '7',
  '8',
  '9',
  '10',
  '11',
  '13',
  '14',
  '15',
  '16',
  '17',
  '18',
  '4',
  '5'],
 '2': ['0', '19', '10', '11', '12', '20', '21', '22', '23', '4', '5'],
 '4': ['0', '3', '24', '25', '21', '26', '17', '5'],
 '5': ['0', '3', '27', '28', '29', '12', '4'],
 '1': ['30', '6', '31', '32', '33', '34', '35', '36', '37', '38'],
 '6': ['1', '30', '39', '40', '41', '31', '42', '32', '33', '34', '35', '36']}

In [5]:
in_degree = {}
for (key, value) in content:
    in_degree.setdefault(value, []).append(key)
in_degree

{'3': ['0', '4', '5'],
 '2': ['0'],
 '4': ['0', '3', '2', '5'],
 '5': ['0', '3', '2', '4'],
 '0': ['3', '2', '4', '5'],
 '7': ['3'],
 '8': ['3'],
 '9': ['3'],
 '10': ['3', '2'],
 '11': ['3', '2'],
 '13': ['3'],
 '14': ['3'],
 '15': ['3'],
 '16': ['3'],
 '17': ['3', '4'],
 '18': ['3'],
 '19': ['2'],
 '12': ['2', '5'],
 '20': ['2'],
 '21': ['2', '4'],
 '22': ['2'],
 '23': ['2'],
 '24': ['4'],
 '25': ['4'],
 '26': ['4'],
 '27': ['5'],
 '28': ['5'],
 '29': ['5'],
 '30': ['1', '6'],
 '6': ['1'],
 '31': ['1', '6'],
 '32': ['1', '6'],
 '33': ['1', '6'],
 '34': ['1', '6'],
 '35': ['1', '6'],
 '36': ['1', '6'],
 '37': ['1'],
 '38': ['1'],
 '1': ['6'],
 '39': ['6'],
 '40': ['6'],
 '41': ['6'],
 '42': ['6']}


2) Using this (or these) inverted index/indices of links, create a function that calculates a dictionary, with node numbers as keys and rank weights as values:

a) initially, all nodes should have the same weight: 1/n, where n is the number of nodes.

In [18]:
rank = {}

In [19]:
def pagerank(out_degree, in_degree):
    node = set(flatten(content))
    n = len(node)
    for each in node:
        rank[each] = 1/n
    #rank = dict.fromkeys(node, 1/n) 
    return rank

In [20]:
rank = pagerank(out_degree, in_degree)
rank

{'0': 0.023255813953488372,
 '26': 0.023255813953488372,
 '22': 0.023255813953488372,
 '20': 0.023255813953488372,
 '11': 0.023255813953488372,
 '38': 0.023255813953488372,
 '39': 0.023255813953488372,
 '15': 0.023255813953488372,
 '25': 0.023255813953488372,
 '5': 0.023255813953488372,
 '36': 0.023255813953488372,
 '28': 0.023255813953488372,
 '18': 0.023255813953488372,
 '29': 0.023255813953488372,
 '10': 0.023255813953488372,
 '21': 0.023255813953488372,
 '33': 0.023255813953488372,
 '4': 0.023255813953488372,
 '17': 0.023255813953488372,
 '24': 0.023255813953488372,
 '42': 0.023255813953488372,
 '12': 0.023255813953488372,
 '3': 0.023255813953488372,
 '2': 0.023255813953488372,
 '30': 0.023255813953488372,
 '6': 0.023255813953488372,
 '31': 0.023255813953488372,
 '7': 0.023255813953488372,
 '40': 0.023255813953488372,
 '35': 0.023255813953488372,
 '13': 0.023255813953488372,
 '8': 0.023255813953488372,
 '1': 0.023255813953488372,
 '9': 0.023255813953488372,
 '37': 0.023255813953488

b) Code one cycle of rank value updates, using a value for p (the probability that a user will go directly to a node rather than following a link to it) of 0.2.

c) Once you have one cycle of rank updates working, alter your function so that it calls itself, feeding the output of one cycle into the next cycle as input, until the rank vector of a cycle is the same as the ranks from the previous cycle. Build in a trace so you can track the number of cycles. How many did it take?

In [21]:
counter = 0

In [22]:
def pagerank(out_degree, in_degree, rank, p):
    global counter
    node = set(flatten(content))
    n = len(node)
    
    check = rank.copy()
    counter += 1
    
    out_amount = {}
    for key, value in out_degree.items():
        out_amount[key] = len(value)
       
    in_amount = {}
    for k, value in in_degree.items():
        for each in value:
            in_amount.setdefault(k, []).append(float(str(rank.get(each)))/int(out_amount.get(each)))
    in_amount = {k: sum(map(float, (i for i in v))) for k, v in in_amount.items()}
    for each in rank.keys():
        if each in in_amount.keys():
            rank[each] = p/n + (1-p) * (in_amount[each])
        else:
            rank[each] = p/n + (1-p) * 0
    for key in rank.keys():
        rank[key] = round(rank[key],7)      
    
    if check != rank:
        print('rank is',rank)
        print('check is',check)
        print('number of cycle:', counter)
        rank = pagerank(out_degree, in_degree, rank, p)
    else:
        return rank, counter
    

In [17]:
rank = pagerank(out_degree, in_degree, rank, .2)
rank

rank is {'0': 0.0126548, '26': 0.0069767, '22': 0.0063425, '20': 0.0063425, '11': 0.0076714, '38': 0.0065116, '39': 0.0062016, '15': 0.0059801, '25': 0.0069767, '5': 0.0146481, '36': 0.008062, '28': 0.007309, '18': 0.0059801, '29': 0.007309, '10': 0.0076714, '21': 0.0086681, '33': 0.008062, '4': 0.0149804, '17': 0.0083056, '24': 0.0069767, '42': 0.0062016, '12': 0.0090003, '3': 0.0142857, '2': 0.0093023, '30': 0.008062, '6': 0.0065116, '31': 0.008062, '7': 0.0059801, '40': 0.0062016, '35': 0.008062, '13': 0.0059801, '8': 0.0059801, '1': 0.0062016, '9': 0.0059801, '37': 0.0065116, '34': 0.008062, '16': 0.0059801, '14': 0.0059801, '23': 0.0063425, '41': 0.0062016, '27': 0.007309, '32': 0.008062, '19': 0.0063425}
check is {'0': 0.023255813953488372, '26': 0.023255813953488372, '22': 0.023255813953488372, '20': 0.023255813953488372, '11': 0.023255813953488372, '38': 0.023255813953488372, '39': 0.023255813953488372, '15': 0.023255813953488372, '25': 0.023255813953488372, '5': 0.023255813953



d) Adjust the value of p to 0.05, and run again. Does the number of cycles change?

In [23]:
second_rank = pagerank(out_degree, in_degree, rank, .05)
second_rank

rank is {'0': 0.0106671, '26': 0.0039244, '22': 0.0031712, '20': 0.0031712, '11': 0.0047493, '38': 0.0033721, '39': 0.0030039, '15': 0.0027409, '25': 0.0039244, '5': 0.0130342, '36': 0.0052132, '28': 0.0043189, '18': 0.0027409, '29': 0.0043189, '10': 0.0047493, '21': 0.0059329, '33': 0.0052132, '4': 0.0134287, '17': 0.0055025, '24': 0.0039244, '42': 0.0030039, '12': 0.0063274, '3': 0.0126038, '2': 0.006686, '30': 0.0052132, '6': 0.0033721, '31': 0.0052132, '7': 0.0027409, '40': 0.0030039, '35': 0.0052132, '13': 0.0027409, '8': 0.0027409, '1': 0.0030039, '9': 0.0027409, '37': 0.0033721, '34': 0.0052132, '16': 0.0027409, '14': 0.0027409, '23': 0.0031712, '41': 0.0030039, '27': 0.0043189, '32': 0.0052132, '19': 0.0031712}
check is {'0': 0.023255813953488372, '26': 0.023255813953488372, '22': 0.023255813953488372, '20': 0.023255813953488372, '11': 0.023255813953488372, '38': 0.023255813953488372, '39': 0.023255813953488372, '15': 0.023255813953488372, '25': 0.023255813953488372, '5': 0.023

### the number of cycles doesn't change a lot